Reference: https://www.kaggle.com/code/ekaterinadranitsyna/segformer-water-segmentation-pytorch/notebook

In [4]:
# !pip install transformers datasets
# !pip install --upgrade sympy
# !pip install --upgrade datasets
# !pip install evaluate
# !pip install transformers
# !pip install torchmetrics

  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
Using cached sympy-1.13.3-py3-none-any.whl (6.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.13.3 which is incompatible.
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3


In [1]:
import os
from datasets import Dataset
from transformers import TrainingArguments, Trainer
import torch
from PIL import Image
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation

processor = SegformerImageProcessor()

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# 데이터 경로 설정
train_images_dir = "/WD/content/car_damage_data_v/train/image"
train_masks_dir = "/WD/content/car_damage_data_v/masks/train"

val_images_dir = "/WD/content/car_damage_data_v/valid/image"
val_masks_dir = "/WD/content/car_damage_data_v/masks/valid"

# 배경 클래스 포함
label_mapping = {'Background': 0, 'Damaged': 1}

# id2label 및 label2id 생성
id2label = {v: k for k, v in label_mapping.items()}  # ID -> Label
label2id = {k: v for k, v in label_mapping.items()}  # Label -> ID

# 출력 확인
print("id2label:", id2label)
print("label2id:", label2id)

# 라벨 개수
num_labels = len(id2label)
print("Number of labels:", num_labels)

/root/anaconda3/envs/car/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda
id2label: {0: 'Background', 1: 'Damaged'}
label2id: {'Background': 0, 'Damaged': 1}
Number of labels: 2


### 이미 학습된 모델 불러오기(teacher model)

In [2]:
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation

# Teacher Model (SegFormer-B3) 로드
teacher_model_name = "nvidia/mit-b3"
processor = SegformerImageProcessor.from_pretrained(teacher_model_name)
teacher_model = SegformerForSemanticSegmentation.from_pretrained(
    "/WD/Segformer-model",
    config="/WD/Segformer-model/config.json"
)
teacher_model.eval()  # Freeze teacher model
for param in teacher_model.parameters():
    param.requires_grad = False

/root/anaconda3/envs/car/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/anaconda3/envs/car/lib/python3.12/site-packages/transformers/utils/deprecation.py:165: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)


### student 모델

In [5]:
# Student Model (SegFormer-B0) 로드
student_model_name = "nvidia/mit-b0"
student_model = SegformerForSemanticSegmentation.from_pretrained(
    student_model_name,
    num_labels=2,  # 동일한 클래스 수
    id2label=id2label,
    label2id=label2id
)
student_model.train()  # 학습 가능하도록 설정

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SegformerForSemanticSegmentation(
  (segformer): SegformerModel(
    (encoder): SegformerEncoder(
      (patch_embeddings): ModuleList(
        (0): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(3, 32, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          (layer_norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        )
        (1): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (2): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(64, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
        )
        (3): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(160, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  

### 파라미터 수 확인

In [31]:
# teacher model
total_params = sum(p.numel() for p in teacher_model.parameters())
trainable_params = sum(p.numel() for p in teacher_model.parameters() if p.requires_grad)

print(f'Total parameters: {total_params}')
print(f'Trainable parameters: {trainable_params}')

Total parameters: 47224002
Trainable parameters: 0


In [32]:
# student model
total_params = sum(p.numel() for p in student_model.parameters())
trainable_params = sum(p.numel() for p in student_model.parameters() if p.requires_grad)

print(f'Total parameters: {total_params}')
print(f'Trainable parameters: {trainable_params}')

Total parameters: 3714658
Trainable parameters: 3714658


### Load Dataset & Preprocessing

In [2]:
def load_data(images_dir, masks_dir):
    images = sorted(os.listdir(images_dir))
    masks = sorted(os.listdir(masks_dir))

    data = []
    for img_name, mask_name in zip(images, masks):
        image_path = os.path.join(images_dir, img_name)
        mask_path = os.path.join(masks_dir, mask_name)
        data.append({"image": image_path, "mask": mask_path})
    return data

train_data = load_data(train_images_dir, train_masks_dir)
val_data = load_data(val_images_dir, val_masks_dir)

# 데이터셋 준비
def preprocess(example):
    image = Image.open(example["image"]).convert("RGB").resize((512, 512))  # 512x512로 축소
    mask = Image.open(example["mask"]).resize((512, 512))
    encoding = processor(image, mask, return_tensors="pt")
    encoding = {k: v.squeeze(0).to(device) for k, v in encoding.items()}
    return encoding


train_dataset = Dataset.from_list(train_data).map(preprocess)
val_dataset = Dataset.from_list(val_data).map(preprocess)

Map: 100%|██████████| 1513/1513 [01:00<00:00, 24.85 examples/s]


In [3]:
# Evaluation Metrics
import torch
from torch import nn
import evaluate

metric = evaluate.load("mean_iou")

def compute_metrics(eval_pred):
  with torch.no_grad():
    logits, labels = eval_pred
    logits_tensor = torch.from_numpy(logits)
    # scale the logits to the size of the label
    logits_tensor = nn.functional.interpolate(
        logits_tensor,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
    ).argmax(dim=1)

    pred_labels = logits_tensor.detach().cpu().numpy()
    metrics = metric.compute(
        predictions=pred_labels,
        references=labels,
        num_labels=len(id2label),
        ignore_index = None,
        reduce_labels=processor.do_reduce_labels,
    )

    # add per category metrics as individual key-value pairs
    per_category_accuracy = metrics.pop("per_category_accuracy").tolist()
    per_category_iou = metrics.pop("per_category_iou").tolist()

    metrics.update({f"accuracy_{id2label[i]}": v for i, v in enumerate(per_category_accuracy)})
    metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})

    return metrics

### loss

In [8]:
# Logits Distillation Loss

import torch.nn.functional as F

def distillation_loss(student_logits, teacher_logits, temperature=2.0):
    """
    KL Divergence Loss for distillation.
    """
    teacher_probs = F.softmax(teacher_logits / temperature, dim=-1)
    student_log_probs = F.log_softmax(student_logits / temperature, dim=-1)
    loss = F.kl_div(student_log_probs, teacher_probs, reduction="batchmean") * (temperature**2)
    return loss


In [9]:
def feature_loss(student_features, teacher_features):
    """
    MSE Loss for feature map distillation.
    """
    loss = 0
    for sf, tf in zip(student_features, teacher_features):
        loss += F.mse_loss(sf, tf)
    return loss

In [10]:
def segmentation_loss(student_output, target):
    """
    Standard cross-entropy loss for segmentation.
    """
    return F.cross_entropy(student_output, target)

### train

In [4]:
# 데이터 처리
train_dataset.set_format(type='torch', columns=['pixel_values', 'labels'])
val_dataset.set_format(type='torch', columns=['pixel_values', 'labels'])

In [5]:
from torch.utils.data import DataLoader

batch_size = 4

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [6]:
# GPU 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# 모델을 GPU로 이동
teacher_model = teacher_model.to(device)
student_model = student_model.to(device)

cuda


NameError: name 'teacher_model' is not defined

In [ ]:
def resize_target(target, size):
    """
    타겟 마스크를 모델 출력 크기에 맞게 리사이즈.
    Args:
        target: [batch_size, height, width] 텐서.
        size: (height, width)로 모델 출력 크기.
    Returns:
        리사이즈된 타겟 마스크 (정수형).
    """
    target = target.unsqueeze(1)  # [B, H, W] -> [B, 1, H, W]
    target = F.interpolate(target.float(), size=size, mode="nearest")  # 리사이즈
    return target.squeeze(1).long()  # 다시 [B, H, W]로 변환하고 정수형으로 변경


### train

In [34]:
import os
import torch
from tqdm import tqdm
from torch.optim import Adam
import evaluate

# mean_iou 메트릭 로드
metric = evaluate.load("mean_iou")

# 로그 파일 설정
log_file = "training_log_distillation.txt"
if os.path.exists(log_file):
    os.remove(log_file)  # 이전 로그 파일 삭제

def write_log(message):
    with open(log_file, "a") as f:
        f.write(message + "\n")
    print(message)

# Optimizer 설정
optimizer = Adam(student_model.parameters(), lr=5e-5)

# 가장 높은 IoU 값을 추적하기 위한 변수
best_val_iou = 0.0  # 초기 최적 IoU 값
best_model_path = "best_model.pth"  # 저장할 모델 파일 경로

# Training Loop
num_epochs = 20
for epoch in range(num_epochs):
    write_log(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss = 0
    train_iou = 0  # IoU 누적 변수

    # tqdm을 사용한 학습 진행 상황 표시
    train_progress_bar = tqdm(train_dataloader, desc="Training", leave=True)

    for batch_idx, batch in enumerate(train_progress_bar):
        optimizer.zero_grad()

        # 데이터 준비
        inputs = batch['pixel_values'].to(device)
        targets = batch['labels'].to(device)

        # 교사 모델 추론
        with torch.no_grad():
            teacher_outputs = teacher_model(pixel_values=inputs)
        teacher_logits = teacher_outputs.logits

        # 학생 모델 추론
        student_outputs = student_model(pixel_values=inputs)
        student_logits = student_outputs.logits

        # 모델 출력 크기 가져오기
        output_size = student_logits.shape[-2:]  # (height, width)

        # 타겟 크기 조정 및 정수형 변환
        resized_targets = resize_target(targets, size=output_size)

        # 손실 계산
        logit_loss = distillation_loss(student_logits, teacher_logits)
        segmentation_loss_value = segmentation_loss(student_logits, resized_targets)

        total_loss = logit_loss + segmentation_loss_value
        total_loss.backward()

        optimizer.step()

        # IoU 계산
        preds = student_logits.argmax(dim=1)  # [batch_size, height, width]
        iou = metric.compute(
            predictions=preds.detach().cpu().numpy(),
            references=resized_targets.detach().cpu().numpy(),
            num_labels=len(id2label),
            ignore_index=None,
            reduce_labels=processor.do_reduce_labels,
        )["mean_iou"]

        # 손실 및 IoU 누적
        train_loss += total_loss.item()
        train_iou += iou

        train_progress_bar.set_postfix({"Batch Loss": total_loss.item(), "Batch mIoU": iou})

    avg_train_loss = train_loss / len(train_dataloader)
    avg_train_iou = train_iou / len(train_dataloader)
    write_log(f"Epoch {epoch + 1}/{num_epochs}, Average Training Loss: {avg_train_loss:.4f}, Average Training mIoU: {avg_train_iou:.4f}")

    # Validation 단계 추가
    student_model.eval()
    val_loss = 0
    val_iou = 0
    val_progress_bar = tqdm(val_dataloader, desc="Validation", leave=True)

    with torch.no_grad():
        for batch_idx, batch in enumerate(val_progress_bar):
            # 데이터 준비
            inputs = batch['pixel_values'].to(device)
            targets = batch['labels'].to(device)

            # 학생 모델 추론
            student_outputs = student_model(pixel_values=inputs)
            student_logits = student_outputs.logits

            # 모델 출력 크기 가져오기
            output_size = student_logits.shape[-2:]

            # 타겟 크기 조정 및 정수형 변환
            resized_targets = resize_target(targets, size=output_size)

            # 손실 계산
            segmentation_loss_value = segmentation_loss(student_logits, resized_targets)

            # IoU 계산
            preds = student_logits.argmax(dim=1)
            iou = metric.compute(
                predictions=preds.detach().cpu().numpy(),
                references=resized_targets.detach().cpu().numpy(),
                num_labels=len(id2label),
                ignore_index=None,
                reduce_labels=processor.do_reduce_labels,
            )["mean_iou"]

            # 손실 및 IoU 누적
            val_loss += segmentation_loss_value.item()
            val_iou += iou

    avg_val_loss = val_loss / len(val_dataloader)
    avg_val_iou = val_iou / len(val_dataloader)
    write_log(f"Epoch {epoch + 1}/{num_epochs}, Average Validation Loss: {avg_val_loss:.4f}, Average Validation mIoU: {avg_val_iou:.4f}")

    # 가장 좋은 IoU 모델 저장
    if avg_val_iou > best_val_iou:
        best_val_iou = avg_val_iou
        torch.save(student_model.state_dict(), best_model_path)
        write_log(f"New best model saved with mIoU: {best_val_iou:.4f}")

    # 모델을 다시 학습 모드로 전환
    student_model.train()


Epoch 1/20


Training: 100%|██████████| 1766/1766 [04:34<00:00,  6.44it/s, Batch Loss=33.3, Batch mIoU=0.511]


Epoch 1/20, Average Training Loss: 58.3323, Average Training mIoU: 0.4283


Validation: 100%|██████████| 379/379 [00:38<00:00,  9.75it/s]


Epoch 1/20, Average Validation Loss: 0.8392, Average Validation mIoU: 0.4200
New best model saved with mIoU: 0.4200
Epoch 2/20


...

Epoch 19/20, Average Validation Loss: 0.1671, Average Validation mIoU: 0.6872
New best model saved with mIoU: 0.6872
Epoch 20/20


Training: 100%|██████████| 1766/1766 [04:18<00:00,  6.83it/s, Batch Loss=13, Batch mIoU=0.708]  


Epoch 20/20, Average Training Loss: 16.4879, Average Training mIoU: 0.7161


Validation: 100%|██████████| 379/379 [00:37<00:00,  9.98it/s]

Epoch 20/20, Average Validation Loss: 0.1764, Average Validation mIoU: 0.6821


### test

In [1]:
# Test
test_images_dir = '/WD/content/car_damage_data_v/test/image'
test_masks_dir = '/WD/content/car_damage_data_v/masks/test'
test_data = load_data(test_images_dir, test_masks_dir)
test_dataset = Dataset.from_list(test_data).map(preprocess)
test_dataset.set_format(type='torch', columns=['pixel_values', 'labels'])

NameError: name 'load_data' is not defined

In [47]:
from torch.utils.data import DataLoader

# Dataset을 DataLoader로 변환
batch_size = 4  # 배치 크기 설정
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)